In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,71,2024-03-25,-4,106.371429,96.785714,0.525171,12.027143,21.495714,0.164729,109.630000,...,11.562857,23.478571,0.184500,116.418571,0,3,0,2,5,2024
1,72,2024-03-25,9,121.154930,96.757746,0.579592,10.273239,24.769014,0.191254,124.535211,...,10.741429,27.480000,0.208729,119.028571,0,3,0,2,2,2024
2,71,2024-03-25,-5,111.428571,99.527143,0.532486,13.162857,24.642857,0.196286,111.524286,...,11.131429,29.375714,0.194300,118.025714,0,3,0,3,9,2024
3,72,2024-03-25,-6,111.098592,96.342254,0.533789,11.049296,24.719718,0.178211,114.605634,...,11.702817,24.416901,0.181127,113.861972,0,3,0,2,3,2024
4,72,2024-03-25,3,117.267606,97.815493,0.569113,12.667606,25.101408,0.228549,119.377465,...,12.664789,22.992958,0.174437,110.897183,0,3,0,2,24,2024
5,72,2024-03-25,-6,107.802817,96.447887,0.509451,12.704225,27.066197,0.190648,110.554930,...,10.862857,24.884286,0.202257,115.288571,0,3,0,2,25,2024
6,72,2024-03-25,2,113.746479,102.115493,0.537859,11.901408,20.256338,0.172028,111.204225,...,10.515493,24.318310,0.188648,115.666197,0,3,0,2,30,2024
7,72,2024-03-25,1,105.929577,97.484507,0.512972,12.863380,22.815493,0.188169,108.095775,...,10.723944,25.484507,0.176014,119.456338,0,3,0,3,15,2024
8,71,2024-03-25,3,118.642857,99.650000,0.563829,10.641429,22.252857,0.197714,119.048571,...,12.873239,28.385915,0.209972,117.098592,0,3,0,4,7,2024
9,71,2024-03-25,-2,114.357143,97.000000,0.528357,10.010000,24.554286,0.227443,117.482857,...,11.225714,23.992857,0.173986,118.005714,0,3,0,3,23,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-25,CHO,CLE,6.000000,1.142857,0
1,2024-03-25,BOS,ATL,1.148148,5.900000,1
2,2024-03-25,DET,NYK,10.500000,1.060606,0
3,2024-03-25,BRK,TOR,1.408163,3.000000,0
4,2024-03-25,PHO,SAS,1.125000,6.500000,1
5,2024-03-25,POR,HOU,6.250000,1.133333,0
6,2024-03-25,WAS,CHI,7.500000,1.100000,0
7,2024-03-25,MEM,DEN,9.500000,1.068966,0
8,2024-03-25,DAL,UTA,1.289855,3.750000,1
9,2024-03-25,PHI,SAC,4.100000,1.253165,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-25,CHO,CLE,6.000000,1.142857,0
1,2024-03-25,BOS,ATL,1.148148,5.900000,0
2,2024-03-25,DET,NYK,10.500000,1.060606,0
3,2024-03-25,BRK,TOR,1.408163,3.000000,0
4,2024-03-25,PHO,SAS,1.125000,6.500000,1
5,2024-03-25,POR,HOU,6.250000,1.133333,0
6,2024-03-25,WAS,CHI,7.500000,1.100000,0
7,2024-03-25,MEM,DEN,9.500000,1.068966,0
8,2024-03-25,DAL,UTA,1.289855,3.750000,0
9,2024-03-25,PHI,SAC,4.100000,1.253165,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-25,CHO,CLE,6.000000,1.142857,0
1,2024-03-25,BOS,ATL,1.148148,5.900000,1
2,2024-03-25,DET,NYK,10.500000,1.060606,0
3,2024-03-25,BRK,TOR,1.408163,3.000000,0
4,2024-03-25,PHO,SAS,1.125000,6.500000,1
5,2024-03-25,POR,HOU,6.250000,1.133333,0
6,2024-03-25,WAS,CHI,7.500000,1.100000,0
7,2024-03-25,MEM,DEN,9.500000,1.068966,0
8,2024-03-25,DAL,UTA,1.289855,3.750000,1
9,2024-03-25,PHI,SAC,4.100000,1.253165,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-25,CHO,CLE,6.000000,1.142857,0
1,2024-03-25,BOS,ATL,1.148148,5.900000,1
2,2024-03-25,DET,NYK,10.500000,1.060606,0
3,2024-03-25,BRK,TOR,1.408163,3.000000,0
4,2024-03-25,PHO,SAS,1.125000,6.500000,1
5,2024-03-25,POR,HOU,6.250000,1.133333,0
6,2024-03-25,WAS,CHI,7.500000,1.100000,0
7,2024-03-25,MEM,DEN,9.500000,1.068966,0
8,2024-03-25,DAL,UTA,1.289855,3.750000,0
9,2024-03-25,PHI,SAC,4.100000,1.253165,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-25,CHO,CLE,6.000000,1.142857,0.0
1,2024-03-25,BOS,ATL,1.148148,5.900000,1.0
2,2024-03-25,DET,NYK,10.500000,1.060606,0.0
3,2024-03-25,BRK,TOR,1.408163,3.000000,0.0
4,2024-03-25,PHO,SAS,1.125000,6.500000,1.0
5,2024-03-25,POR,HOU,6.250000,1.133333,0.0
6,2024-03-25,WAS,CHI,7.500000,1.100000,1.0
7,2024-03-25,MEM,DEN,9.500000,1.068966,0.0
8,2024-03-25,DAL,UTA,1.289855,3.750000,1.0
9,2024-03-25,PHI,SAC,4.100000,1.253165,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-25,CHO,CLE,6.000000,1.142857,0.0
1,2024-03-25,BOS,ATL,1.148148,5.900000,1.0
2,2024-03-25,DET,NYK,10.500000,1.060606,0.0
3,2024-03-25,BRK,TOR,1.408163,3.000000,0.0
4,2024-03-25,PHO,SAS,1.125000,6.500000,1.0
5,2024-03-25,POR,HOU,6.250000,1.133333,0.0
6,2024-03-25,WAS,CHI,7.500000,1.100000,1.0
7,2024-03-25,MEM,DEN,9.500000,1.068966,0.0
8,2024-03-25,DAL,UTA,1.289855,3.750000,1.0
9,2024-03-25,PHI,SAC,4.100000,1.253165,0.0
